# Wavespectra tutorial

## Links

### Github repository:
https://github.com/wavespectra/wavespectra

### Documentation:
https://wavespectra.readthedocs.io/en/latest/

## Installation

### Pre-install dependencies

* Linux, Windows and MacOS
* python>=3.8

### Installing with pip

    pip install wavespectra

or alternatively:

    pip install wavespectra[extra]

for a complete installation including some extra libraries such as netcdf and zarr.

### Installing with conda

    conda install -c conda-forge wavespectra

## Introduction

Wavespectra is an open source project for processing ocean wave spectral data. The library is built on top of xarray and focussed on speed and efficiency for large numbers of spectra.

### Xarray

[xarray](https://docs.xarray.dev/en/stable/) is a very useful python library designed to handle labelled multi-dimensional arrays in an intuitive way.

In [2]:
from pathlib import Path
import xarray as xr

datadir = Path("../data")

dset = xr.open_dataset(datadir / "ww3file.nc", engine="netcdf4")
dset

<xarray.Dataset> Size: 44kB
Dimensions:    (direction: 24, time: 9, station: 2, frequency: 25)
Coordinates:
  * direction  (direction) float32 96B 90.0 75.0 60.0 45.0 ... 135.0 120.0 105.0
  * frequency  (frequency) float32 100B 0.04118 0.0453 0.04983 ... 0.3687 0.4056
  * station    (station) int32 8B 1 2
  * time       (time) datetime64[ns] 72B 2014-12-01 ... 2014-12-05
Data variables:
    dpt        (time, station) float32 72B ...
    efth       (time, station, frequency, direction) float32 43kB ...
    latitude   (time, station) float32 72B ...
    longitude  (time, station) float32 72B ...
    wnd        (time, station) float32 72B ...
    wnddir     (time, station) float32 72B ...

One of the core objects in xarray is a `Dataset` which is composed by labelled attributes, dimensions, coordinates and data variables, resambling the netcdf data model. The data variables are `DataArray` objects which are multi-dimensional arrays that can share a set of dimensions and coordinates in the Dataset.

These labels can be used to define operations on the data in a very simple but powerful way. 

In [3]:
# Selecting single spectrum from the dataset

dset.sel(time="2014-12-01 00:00", station=1, drop=True)

<xarray.Dataset> Size: 3kB
Dimensions:    (direction: 24, frequency: 25)
Coordinates:
  * direction  (direction) float32 96B 90.0 75.0 60.0 45.0 ... 135.0 120.0 105.0
  * frequency  (frequency) float32 100B 0.04118 0.0453 0.04983 ... 0.3687 0.4056
Data variables:
    dpt        float32 4B ...
    efth       (frequency, direction) float32 2kB ...
    latitude   float32 4B ...
    longitude  float32 4B ...
    wnd        float32 4B ...
    wnddir     float32 4B ...

In [4]:
# Accessing single variable in the Dataset

dset.wnd

<xarray.DataArray 'wnd' (time: 9, station: 2)> Size: 72B
[18 values with dtype=float32]
Coordinates:
  * station  (station) int32 8B 1 2
  * time     (time) datetime64[ns] 72B 2014-12-01 ... 2014-12-05
Attributes:
    long_name:      wind speed at 10m
    standard_name:  wind_speed
    globwave_name:  wind_speed
    units:          m s-1
    valid_min:      0.0
    valid_max:      100.0
    content:        TX
    associates:     time station

In [5]:
# Applying operations across dimensions

dset.wnd.mean()

<xarray.DataArray 'wnd' ()> Size: 4B
array(4.765337, dtype=float32)

### Wavespectra

Wavespectra defines two main objects to extend xarray's Dataset and DataArray:

`Dataset` --> `SpecDataset`

`DataArray` -- > `SpecArray`

These objects (and their bounded methods) are accessed from the **.spec** namespace defined by wavespectra:

In [6]:
from wavespectra import read_ww3

dset = read_ww3(datadir / "ww3file.nc")

dset.spec

<SpecDataset> Size: 44kB
Dimensions:  (time: 9, site: 2, freq: 25, dir: 24)
Coordinates:
  * freq     (freq) float32 100B 0.04118 0.0453 0.04983 ... 0.3352 0.3687 0.4056
  * site     (site) int32 8B 1 2
  * time     (time) datetime64[ns] 72B 2014-12-01 ... 2014-12-05
  * dir      (dir) float32 96B 270.0 255.0 240.0 225.0 ... 315.0 300.0 285.0
Data variables:
    dpt      (time, site) float32 72B dask.array<SpecDataset>
    efth     (time, site, freq, dir) float32 43kB dask.array<SpecDataset>
    lat      (site) float32 8B dask.array<SpecDataset>
    lon      (site) float32 8B dask.array<SpecDataset>
    wspd     (time, site) float32 72B dask.array<SpecDataset>
    wdir     (time, site) float32 72B dask.array<SpecDataset>

### conventions

Wavespectra defines some conventions around the units and names for some variables and spectral dimensions:

* **freq** (Hz)
* **dir** (degree, coming_from)
* **efth** (m2s/degree)
* **wspd** (m/s)
* **wdir** (degree, coming_from)

### Interface

The spec namespace provides an interface for processing wave spectra data:

* Reading from and writing to several wave spectra formats
* Calcule spectral statistics
* Spectral partitioning
* Plotting
* Selecting from pointcloud datasets
* Spectra reconstruction (under development)